<div style="font-size: 25px">
Let's use one of the most realistic retail time-series dataset, i.e. Walmart data. <br>
<br>
It was made available for a Kaggle competition that you can check out [here](https://www.kaggle.com/c/m5-forecasting-accuracy). <br>
<br>
The original format of the data was in a "wide" format to made it smaller in memory, but that doesn't really work too well with databases, and we don't see that very often in the real world. I added a date column to replace the date identifier columns that were previously there, and I made the data smaller by only subsetting to the state of Texas. <br>

PS: Set `sampled` in the cells below to `True` to test on a subset!
</div>

In [9]:
!jupyter --config-dir

/Users/JPB/.jupyter


In [29]:
!pip install --upgrade jupyterthemes -q
!jt -t oceans16 -tf merriserif -tfs 14 -nf ptsans -nfs 14
# styles chesterish, grade3, ggruvboxd, gruvboxl, monokai, oceans16, onedork, solarizedd, solarizedl

In [1]:
# to get Jupyter display all the variables values, not only the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [3]:
from platform import python_version
print('Python version:', python_version())
print(pd.__version__)
print(np.__version__)
import matplotlib
print(matplotlib.__version__)
import statsmodels
print(statsmodels.__version__)
print(sns.__version__)
import sklearn
print(sklearn.__version__)

Python version: 3.11.4
1.5.3
1.25.0
3.7.1
0.14.0
0.12.2
1.2.2


In [11]:
from pathlib import Path  # pathlib is an OOP filesystem paths https://docs.python.org/3/library/pathlib.html
datadir = Path('data')
data = datadir / 'data.zip'

if not Path(datadir).exists():
    datadir.mkdir()


In [8]:
!pip install -Uq --no-cache-dir gdown --pre
!gdown --no-cookies  -O $data --fuzzy  https://drive.google.com/file/d/1wqGP6UuGh5wxd2LBTBNtVqvHBkLhhAmO/view?usp=sharing
!unzip $data
!ls -al $datadir

Downloading...
From (uriginal): https://drive.google.com/uc?id=1wqGP6UuGh5wxd2LBTBNtVqvHBkLhhAmO
From (redirected): https://drive.google.com/uc?id=1wqGP6UuGh5wxd2LBTBNtVqvHBkLhhAmO&confirm=t&uuid=d03abe8e-a78f-414c-9770-15a6ac2fd26e
To: /Volumes/DATA/Dropbox/IMAC BACKUP/WORK/PROJECTS/PORTFOLIO/TIME_SERIES/data/data.zip
100%|████████████████████████████████████████| 179M/179M [00:06<00:00, 27.5MB/s]
Archive:  data/data.zip
replace data/sales_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
total 4370784
drwxr-xr-x   9 JPB  admin         306 Jun 30 22:01 .
drwxr-xr-x  12 JPB  admin         408 Jun 30 22:03 ..
-rw-r--r--@  1 JPB  admin        6148 Jun 29 00:47 .DS_Store
-rw-r--r--   1 JPB  admin           0 Sep  3  2017 .gitkeep
-rw-r--r--   1 JPB  admin   179336037 Jun 30 22:01 data.zip
-rw-r--r--   1 JPB  admin   928535251 Jan 23 09:51 sales_data.csv
-rw-r--r--   1 JPB  admin     1853101 Mar 11 16:25 sales_data_sampled.csv
-rw-r--r--   1 JPB  admin  1118640783 Jan 23 09:46 transaction

In [9]:
sampled = True  # select true to test on a smaller dataset
path_suffix = '' if not sampled else '_sampled'

## Getting our data in the right format

<div style="font-size: 25px">Time-series data has to be collected from some real-world, data-generating process. That means that raw data comes in as a series of observations. Depending on your experience with time-series data, you may be used to data that looks like this:<br>
<br>
| Date       | Sales |<br>
|------------|-------|<br>
| 2022-01-01 |  23   |<br>
| 2022-01-02 |  45   |<br>
| 2022-01-03 |  12   |<br>
| 2022-01-04 |  67   |<br>
| 2022-01-05 |  89   |<br>
<br>
But, if you're in retail, each of those "sales" probably came in some JSON from some point-of-sale system (i.e. cash register) that probably looked something like this:<br>

```
{
    "timestamp": 2022-01-01 12:34:56,
    "product_id": 5,
    "store_id": 12,
    "category_id": 36,
    ...
}
```
<br>
Usually, it's the job of a data engineer to collect all of these records and aggregate them into a nice, tabular format, but it's worth at least having an appreciation for how it's done. So, we're going to start from a mock version of a transactions table.<br>

In [12]:
transactions = pd.read_csv(f'{datadir}/transactions_data{path_suffix}.csv')

transactions.head()

,date,id,item_id,dept_id,cat_id,store_id,state_id
0,2013-01-01 13:41:03,HOBBIES_1_004_TX_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_1,TX
1,2013-01-01 07:30:52,HOBBIES_1_004_TX_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_1,TX
2,2013-01-01 11:17:38,HOBBIES_1_004_TX_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_1,TX
3,2013-01-01 06:07:58,HOBBIES_1_004_TX_2_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_2,TX
4,2013-01-01 21:51:07,HOBBIES_1_004_TX_2_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_2,TX


In [23]:
transactionsFull = pd.read_csv(f'{datadir}/transactions_data.csv')  # FULL DATASET

transactionsFull.head()
del transctionFull

,date,id,item_id,dept_id,cat_id,store_id,state_id
0,2013-01-01 13:41:03,HOBBIES_1_004_TX_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_1,TX
1,2013-01-01 07:30:52,HOBBIES_1_004_TX_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_1,TX
2,2013-01-01 11:17:38,HOBBIES_1_004_TX_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_1,TX
3,2013-01-01 20:18:59,HOBBIES_1_025_TX_1_evaluation,HOBBIES_1_025,HOBBIES_1,HOBBIES,TX_1,TX
4,2013-01-01 21:36:09,HOBBIES_1_028_TX_1_evaluation,HOBBIES_1_028,HOBBIES_1,HOBBIES,TX_1,TX


In [13]:
transactions.dtypes

date        object
id          object
item_id     object
dept_id     object
cat_id      object
store_id    object
state_id    object
dtype: object

<span style="font-size: 25px"> We can see that this is a DataFrame where each row relates to purchases for an individual item.

- `date`: the time at which an item was bought, down to the second
- `id`: the product ID. Each of these is an individual item at a specific store.
- `item_id`: this is an identifier for items, but not at the store level. You can use this to find the same item at different stores.
- `dept_id`: department ID. One level up from `item_id` in the hierarchy
- `cat_id`: category ID. One level up from `dept_id` in the hierarchy
- `store_id`: identifies the specific store where the product was bought
- `state_id`: identifies the specific state where the product was bought

`date` is supposed to be a `datetime`-like object, but we can see that it was loaded in as a string. Let's convert that column to `datetime`.
</span>

In [27]:
transactions['date'] = pd.to_datetime(transactions.date) #, format="%Y-%m-%d %H:%M:%S")
transactions.dtypes

date        datetime64[ns]
id                  object
item_id             object
dept_id             object
cat_id              object
store_id            object
state_id            object
dtype: object